In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [6]:
def sequential_generator(filename, batch_size, check:
                         'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None):
    
    f = open(filename)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, 2))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            
        yield ([batch_features_ft], batch_labels)

In [11]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

    return (train_sentences, train_labels, dev_sentences, dev_labels)


In [ ]:
filetrain='/home/jindal/notebooks/jindal/NER/language_model/combined_train.csv'
filedev='/home/jindal/notebooks/jindal/NER/language_model/combined_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels = train_dev_sentences(filetrain=filetrain,
                   filedev=filedev, check=2)

In [ ]:
print(len(dev_sentences))

In [ ]:
dev_sentences[0]

In [ ]:
print(len(dev_labels))

In [ ]:
n_labels =2

In [7]:
def compile_model():
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [ ]:
print(dev_sentences[1])

In [8]:
def test_model(generator, train_sentences, devLabels, number_of_tests,number_of_epochs, filename_to_log, 
               filename_to_save_weigths, batch_size, train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', pos_label:'only if binary f1',
               load_model_weights=False,model_weights_file:'give filepath as str'=None, 
               nb_sequence_length = nb_sequence_length, nb_embedding_dims= nb_embedding_dims, check_for_generator=None ):
    
    f = open(filename_to_log,"w")
    
    
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model()

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)


        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        # checkpoint = ModelCheckpoint('best_classification_model_million_post_corpus.h5', monitor='val_acc',save_best_only = True, 
    #                                  save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(train_file, batch_size, check_for_generator), 
                steps_per_epoch=steps_per_epoch, epochs=1,
        #         validation_data = sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv', batch_size),
        #         validation_steps = math.ceil(len(dev_sentences) / batch_size),
        #         callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = model.predict(testset_features)


            idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [ ]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 50
combined_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_combined.txt' 
combined_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_combined.h5'
batch_size=32
combined_train_file='/home/jindal/notebooks/jindal/NER/language_model/combined_train.csv'
f1_measure='binary'
pos_label=1
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [ ]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, filename_to_log=combined_pretraining_log,
           filename_to_save_weigths=combined_pretraining_save_weigths,
           batch_size=batch_size,
           train_file=combined_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           nb_sequence_length=nb_sequence_length,
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

# TRANSFER LEARNING HERE


In [9]:
n_labels =2

In [12]:
filetrain = '/home/gwiedemann/notebooks/OffLang/sample_train.txt'
filedev = '/home/gwiedemann/notebooks/OffLang/sample_dev.txt'
train_sentences, train_labels, dev_sentences, dev_labels = train_dev_sentences(filetrain=filetrain,
                   filedev=filedev, check=3)

In [13]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [14]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
combined_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_combined.txt' 
combined_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_combined.h5'
batch_size=32
combined_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_combined.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [ ]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=combined_tl_log, 
           filename_to_save_weigths=combined_tl_save_weigths,
           batch_size=batch_size,
           train_file=combined_tl_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           model_weights_file = model_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 34s 256ms/step - loss: 0.6604 - acc: 0.7060
model saved. F1 is 0.544601
Test-Data: Prec: 0.730, Rec: 0.434, F1: 0.545, Acc: 0.760
Epoch: 2
Epoch 1/1
132/132 [==============================] - 32s 241ms/step - loss: 0.5137 - acc: 0.7588
model saved. F1 is 0.649706
Test-Data: Prec: 0.680, Rec: 0.622, F1: 0.650, Acc: 0.778
Epoch: 3
Epoch 1/1
132/132 [==============================] - 32s 240ms/step - loss: 0.4732 - acc: 0.7744
model saved. F1 is 0.676864
Test-Data: Prec: 0.691, Rec: 0.663, F1: 0.677, Acc: 0.791
Epoch: 4
Epoch 1/1
132/132 [==============================] - 31s 239ms/step - loss: 0.4399 - acc: 0.7952
model saved. F1 is 0.686792
Test-Data: Prec: 0.692, Rec: 0.682, F1: 0.687, Acc: 0.795
Epoch: 5
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.4117 - acc: 0.8151
Test-Data: Prec: 0.684, Rec: 0.682, F1: 0.683, Acc: 0.791
Epoch: 6
Epoch 1/1
132/132 [==============================]